In [ ]:
#dependencies and setup
import pandas as pd

#load files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

#read files
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)


In [ ]:
#combine data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how ="left", on=["school_name", "school_name"])

In [ ]:
#output the data (did this as a test in office hours)
school_data_complete


In [ ]:
#calculate total number of schools
total_schools = len(school_data_complete["school_name"].unique())

#calculate the total number of students
total_students = len(school_data_complete["student_name"])

#calculate the total budget
total_budget = school_data["budget"].sum()

#calculate the average math score
avg_math = school_data_complete["math_score"].mean()

#calculate the average reading score
avg_read = school_data_complete["reading_score"].mean()

#calculate the percent of students with a passing math score
school_data_complete["passing_math"] = school_data_complete["math_score"] >= 70
percent_pass_math = ((school_data_complete["passing_math"].mean())*100)

#calculate the percent of students with a passing reading score
school_data_complete["passing_reading"] = school_data_complete["reading_score"] >= 70
percent_pass_reading = ((school_data_complete["passing_reading"].mean())*100)

#calculate the percent of students who passed both math and reading
pass_both = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)]
overall_pass_count = pass_both["student_name"].count()
overall_pass_percent = overall_pass_count/ total_students * 100


In [ ]:
#print the data
district_summary_df = pd.DataFrame({"Total Schools": total_schools, "Total Students": total_students,
                                    "Total Budget": total_budget, "Average Math Score": avg_math, 
                                    "Average Reading Score": avg_read, "% Passing Math": percent_pass_math, "% Passing Reading": percent_pass_reading, "% Overall Passing": overall_pass_percent}, index=[0])
district_summary_df.head()

In [ ]:
#create dataframe to hold results; for some reason I can't figure out how to only get the school names only written onc

school_summary_df = pd.DataFrame({"School Name": school_data_complete["school_name"], 
                                 "School Type": school_data_complete["type"], "Total Students": school_data_complete["size"], 
                                 "Total School Budget": school_data_complete["budget"]})  

#set the index to be school name
school_summary_df = school_summary_df.set_index("School Name")
school_summary_df = school_summary_df.sort_values(by = "School Name")
 
#find the per student budget
school_summary_df["Per Student Budget"] = school_summary_df["Total School Budget"]/school_summary_df["Total Students"]

#find the average math and reading scores
school_summary_mean_df = school_data_complete.groupby(["school_name"]).mean()
school_summary_df["Average Math Score"] = school_summary_mean_df["math_score"]
school_summary_df["Average Reading Score"] = school_summary_mean_df["reading_score"]

# Calculate the total student count
per_school_counts = school_data_complete["school_name"].value_counts()

# Calculate the passing scores
per_school_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]
per_school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]

# Calculate the number of students passing math and reading by school
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percent of passing math and reading scores per school
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)
                                               & (school_data_complete["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

#find the percent overall passing
school_summary_df["% Overall Passing"] = per_passing_math_reading/ per_school_counts * 100


school_summary_df.head()
school_summary_df

In [ ]:
#find the top performing schools
top_performing_schools = school_summary_df.sort_values(by = "% Overall Passing", ascending = False)
top_performing_schools.head()

In [ ]:
#find the bottom performing schools
bottom_performing_schools = school_summary_df.sort_values(by = "% Overall Passing", ascending = True)
bottom_performing_schools.head()

In [ ]:
#group math scores by grade
grade9_df = school_data_complete.loc[school_data_complete["grade"] == "9th"].groupby(["school_name"])["math_score"].mean()
grade10_df = school_data_complete.loc[school_data_complete["grade"] == "10th"].groupby(["school_name"])["math_score"].mean()
grade11_df = school_data_complete.loc[school_data_complete["grade"] == "11th"].groupby(["school_name"])["math_score"].mean()
grade12_df = school_data_complete.loc[school_data_complete["grade"] == "12th"].groupby(["school_name"])["math_score"].mean()

#create dataframe
math_grade_df = pd.DataFrame({"9th": grade9_df,
      "10th": grade10_df,
      "11th": grade11_df,
      "12th": grade12_df})

math_grade_df

In [ ]:
#group reading scores by grade
grade9_df = school_data_complete.loc[school_data_complete["grade"] == "9th"].groupby(["school_name"])["reading_score"].mean()
grade10_df = school_data_complete.loc[school_data_complete["grade"] == "10th"].groupby(["school_name"])["reading_score"].mean()
grade11_df = school_data_complete.loc[school_data_complete["grade"] == "11th"].groupby(["school_name"])["reading_score"].mean()
grade12_df = school_data_complete.loc[school_data_complete["grade"] == "12th"].groupby(["school_name"])["reading_score"].mean()

#create dataframe
reading_grade_df = pd.DataFrame({"9th": grade9_df,
      "10th": grade10_df,
      "11th": grade11_df,
      "12th": grade12_df})

reading_grade_df

In [ ]:
#create bins and labels for school spending
school_spending_bins = [0, 585, 630, 645, 680]
school_spending_ranges = ["<$585", "$585-630", "$630-645", "$645-680"]


In [ ]:
# categorize spending based on bins
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], school_spending_bins, labels= school_spending_ranges)
school_summary_df

In [ ]:
#create group based off bins
school_summary_df = school_summary_df.groupby("Spending Ranges (Per Student)")
school_summary_df.max()

In [ ]:
#create bins and labels for school size
school_size_bins = [0, 1000, 2000, 5000]
school_size_ranges = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

In [ ]:
#categorize spending based on bins
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], school_size_bins, labels=school_size_ranges, include_lowest = True)
school_summary_df

In [ ]:
#create group based off bins
school_summary_df = school_summary_df.groupby("School Size")
school_summary_df.max()

In [ ]:
#group data based off school type
school_summary_df = school_summary_df.groupby("School Type")
school_summary_df

In [ ]:
#report
#From this data, I can conclude that the school size has an effect on the average scores for reading and math, as well as the percentages of students who are passing reading and math. The larger schools have a significantly lower overall passing percentage compared to the smaller schools, which comes from having lower reading and math averages.
#From grouping the scores by school type, it is clear that Charter Schools tend to have higher math and reading score averages as well as the percentages of students who are passing math and reading, which leads to a much higher overall percent of students passing.
#From looking at the spending ranges per student, we can conclude that even though the larger schools have a higher spending rate per student, that doesn't equate to higher test scores, as the spending rates per student that are lower have the higher test scores in math, reading, and much higher percentages of students who are passing math and reading.